In [2]:
import json
from pathlib import Path

result_path = "~/Downloads/proverbench_pipeline_results.json"
with open(Path(result_path).expanduser(), "r") as f:
    result_json = json.load(f)

from natsort import natsorted
result_json = natsorted(result_json, key=lambda x: x["problem_id"])

In [6]:
result_json[0].keys()

dict_keys(['problem_id', 'dataset', 'problem_dir', 'complete_proof_path', 'complete_fixed_proof', 'original_verification_pass', 'synthesized_verification_pass', 'num_steps', 'detailed_steps', 'processing_time_seconds', 'status', 'timestamp'])

In [22]:
result_json[0]["original_verification_pass"]

False

In [23]:
result_json[0]["synthesized_verification_pass"]

False

## stepwise data

In [10]:
unigrams = ["norm_num", "linarith", "nlinarith", "omega", "ring", "ring_nf", "simp", "simpa", "field_simp", "positivity", "norm_cast"]

In [20]:
skipped = []

for bigbench in result_json:

    # skip big benchmark level decomposition error
    if "detailed_steps" not in bigbench.keys():
        skipped.append(bigbench["problem_id"])
        continue
        
    for smallbench in bigbench["detailed_steps"]:
        sline = []
        # add basic columns
        sline.append(bigbench["problem_id"])
        sline.append(smallbench["step_id"])
        sline.append(1 if smallbench["filled_verification_pass"] else 0)
        if smallbench["hole_verification_pass"]:
            # valid sketch
            sline.append(1)
            sline.append(1 if len(smallbench["additional_info"]["successful_tactics"])>0 else 0)
            # add unigram columns
            for u in unigrams:
                sline.append(1 if u in smallbench["additional_info"]["successful_tactics"] else 0)
        else:
            # invalid sketch, set all other columns to -1
            sline.append(0)
            sline.append(-1)
            for u in unigrams:
                sline.append(-1)
        print(", ".join([f"{p}" for p in sline]))

abstract_algebra__p1, step_0001, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1
abstract_algebra__p1, step_0002, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1
abstract_algebra__p1, step_0003, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1
abstract_algebra__p1, step_0004, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step_0005, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step_0006, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1
abstract_algebra__p1, step_0007, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1
abstract_algebra__p1, step_0008, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1
abstract_algebra__p1, step_0009, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1
abstract_algebra__p1, step_0010, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step_0011, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step_0012, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step_0013, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
abstract_algebra__p1, step

In [21]:
skipped

['abstract_algebra__p2',
 'abstract_algebra__p6',
 'abstract_algebra__p8',
 'abstract_algebra__p22',
 'abstract_algebra__p25',
 'abstract_algebra__p29',
 'aime_2024ii_p7',
 'aime_2025ii_p4',
 'calculus__p1',
 'calculus__p7',
 'calculus__p10',
 'calculus__p14',
 'calculus__p60',
 'calculus__p68',
 'complex_analysis__p1',
 'complex_analysis__p3_2',
 'elementary_algebra__p9',
 'elementary_algebra__p17',
 'elementary_algebra__p18',
 'linear_algebra__p10_1',
 'number_theory__p5',
 'number_theory__p12',
 'number_theory__p15',
 'number_theory__p17',
 'number_theory__p23',
 'probability__p2_3',
 'real_analysis__p6_3',
 'real_analysis__p12_3',
 'real_analysis__p13_3']

## full benchmark data

In [25]:
suite = "proverbench@1"
for bigbench in result_json:
    bline = []
    bline.append(suite)
    bline.append(bigbench["problem_id"])
    bline.append(1 if bigbench["original_verification_pass"] else 0)
    if "detailed_steps" in bigbench.keys():
        hps = [p["hole_verification_pass"] for p in bigbench["detailed_steps"]]
        if False in hps:
            # there's invalid sketch somewhere in the steps
            bline.append(-1)
        else:
            bline.append(1 if bigbench["synthesized_verification_pass"] else 0)
    else:
        # there's full benchmark level bug
        bline.append(-2)
    
    print(", ".join([f"{p}" for p in bline]))
    

proverbench@1, abstract_algebra__p1, 0, -1
proverbench@1, abstract_algebra__p2, 0, -2
proverbench@1, abstract_algebra__p3, 0, -1
proverbench@1, abstract_algebra__p4_1, 0, 0
proverbench@1, abstract_algebra__p4_2, 0, 0
proverbench@1, abstract_algebra__p4_3, 0, -1
proverbench@1, abstract_algebra__p5, 0, -1
proverbench@1, abstract_algebra__p6, 0, -2
proverbench@1, abstract_algebra__p7, 0, -1
proverbench@1, abstract_algebra__p8, 0, -2
proverbench@1, abstract_algebra__p9, 0, -1
proverbench@1, abstract_algebra__p10, 0, -1
proverbench@1, abstract_algebra__p11_1, 1, 0
proverbench@1, abstract_algebra__p11_2, 1, 0
proverbench@1, abstract_algebra__p11_3, 0, -1
proverbench@1, abstract_algebra__p12, 0, -1
proverbench@1, abstract_algebra__p13, 1, 0
proverbench@1, abstract_algebra__p14, 0, -1
proverbench@1, abstract_algebra__p15, 0, -1
proverbench@1, abstract_algebra__p16, 0, -1
proverbench@1, abstract_algebra__p17, 1, -1
proverbench@1, abstract_algebra__p18, 0, -1
proverbench@1, abstract_algebra__p19